# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770400614140                   -0.52    9.0    196ms
  2   -2.771691532850       -2.89       -1.32    1.0   91.5ms
  3   -2.771714368878       -4.64       -2.46    1.0   91.5ms
  4   -2.771714636277       -6.57       -3.14    1.0   94.8ms
  5   -2.771714713669       -7.11       -3.91    2.0    112ms
  6   -2.771714715104       -8.84       -4.59    1.0   97.7ms
  7   -2.771714715248       -9.84       -5.30    2.0    118ms
  8   -2.771714715249      -11.96       -5.92    1.0    105ms
  9   -2.771714715250      -12.47       -6.44    2.0    120ms
 10   -2.771714715250      -13.83       -7.32    1.0    124ms
 11   -2.771714715250   +  -14.57       -8.11    1.0    107ms


-0.00013457327512523073

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770530575267                   -0.52    9.0    1.98s
  2   -2.771779862804       -2.90       -1.32    1.0    151ms
  3   -2.771801651835       -4.66       -2.46    1.0    107ms
  4   -2.771802003929       -6.45       -3.18    1.0   93.0ms
  5   -2.771802074140       -7.15       -4.11    2.0    124ms
  6   -2.771802074373       -9.63       -4.43    1.0   97.5ms
  7   -2.771802074466      -10.03       -5.04    1.0    100ms
  8   -2.771802074475      -11.05       -5.64    1.0    107ms
  9   -2.771802074476      -11.94       -6.35    2.0    125ms
 10   -2.771802074476      -13.42       -6.90    1.0    115ms
 11   -2.771802074476   +  -13.95       -7.77    1.0    108ms
 12   -2.771802074476      -13.83       -8.00    2.0    137ms


0.017612217891823278

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457327512523073
Displaced dipole:  0.017612217891823278
Polarizability :   1.7746791166948508


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920632598e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551058780e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852763806234e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593773007e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787561107e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.274665294057e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.029158597206e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.029158597206e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.655425886572e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.222980651628e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.823834434245e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.218111692550e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.218111692550e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.